In [1]:
# from Pytorch_VGAE import CustomGraphDataset,VGAEModel
import pandas as pd
import torch.nn.functional as F
from torch_geometric.utils import from_scipy_sparse_matrix, dense_to_sparse
from torch_geometric.loader import DataLoader
import numpy as np
from VGAE import VGAEEncoder,prepare_data
from torch_geometric.nn import VGAE
import torch

In [2]:
A_list = []
X_list = []
for i in range(5000):
    A = pd.read_csv(f'../../VGAE_dataset/raw/A/A_{i}.csv', header=None).values
    A = A+A.T
    A_list.append(A)
    X_list.append(pd.read_csv(f'../../VGAE_dataset/raw/X/X_{i}.csv', header=None).values)
    
data_list = prepare_data(X_list, A_list)
loader = DataLoader(data_list, batch_size=1, shuffle=True)

/opt/miniconda3/envs/MindSpore/lib/python3.9/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'train_test_split_edges' is deprecated, use 'transforms.RandomLinkSplit' instead
  warnings.warn(out)


In [3]:
index =3
print(A_list[index].shape,X_list[index].shape)
print(data_list[index].x.shape)

(26, 26) (26, 22)
torch.Size([26, 22])


In [14]:
for batch in loader:
    # print("test_pos_edge_index:", batch.test_pos_edge_index)
    # print("test_neg_edge_index:", batch.test_neg_edge_index)
    
    # 检查 test_pos_edge_index 和 test_neg_edge_index 是否为空
    if batch.test_pos_edge_index.size(1) == 0:
        print("警告: test_pos_edge_index 为空")
        print(batch)
    if batch.test_neg_edge_index.size(1) == 0:
        print("警告: test_neg_edge_index 为空")
    
    # 继续执行其他逻辑

    

警告: test_pos_edge_index 为空
DataBatch(x=[12, 22], val_pos_edge_index=[2, 0], test_pos_edge_index=[2, 0], train_pos_edge_index=[2, 18], train_neg_adj_mask=[12, 12], val_neg_edge_index=[2, 0], test_neg_edge_index=[2, 0], batch=[12], ptr=[2])
警告: test_neg_edge_index 为空
警告: test_pos_edge_index 为空
DataBatch(x=[12, 22], val_pos_edge_index=[2, 0], test_pos_edge_index=[2, 0], train_pos_edge_index=[2, 18], train_neg_adj_mask=[12, 12], val_neg_edge_index=[2, 0], test_neg_edge_index=[2, 0], batch=[12], ptr=[2])
警告: test_neg_edge_index 为空


In [12]:
model = VGAE(encoder=VGAEEncoder(in_channels=22, hidden_channels=32, out_channels=8, num_layers=3))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
epochs = 100

In [5]:
for batch in loader:
    print("Num nodes in batch.x:", batch.x.size(0))
    print("Max train_pos_edge_index:", batch.train_pos_edge_index.max().item())
    # 如果 batch.x 的尺寸不足，可能是数据预处理或批处理的问题

Num nodes in batch.x: 13
Max train_pos_edge_index: 12
Num nodes in batch.x: 29
Max train_pos_edge_index: 28
Num nodes in batch.x: 17
Max train_pos_edge_index: 15
Num nodes in batch.x: 28
Max train_pos_edge_index: 27
Num nodes in batch.x: 29
Max train_pos_edge_index: 27
Num nodes in batch.x: 13
Max train_pos_edge_index: 11
Num nodes in batch.x: 25
Max train_pos_edge_index: 24
Num nodes in batch.x: 29
Max train_pos_edge_index: 28
Num nodes in batch.x: 13
Max train_pos_edge_index: 12
Num nodes in batch.x: 24
Max train_pos_edge_index: 23
Num nodes in batch.x: 22
Max train_pos_edge_index: 21
Num nodes in batch.x: 27
Max train_pos_edge_index: 26
Num nodes in batch.x: 24
Max train_pos_edge_index: 23
Num nodes in batch.x: 15
Max train_pos_edge_index: 14
Num nodes in batch.x: 24
Max train_pos_edge_index: 23
Num nodes in batch.x: 16
Max train_pos_edge_index: 15
Num nodes in batch.x: 13
Max train_pos_edge_index: 12
Num nodes in batch.x: 30
Max train_pos_edge_index: 29
Num nodes in batch.x: 20
Max

In [15]:
def train():
    model.train()  # 设置模型为训练模式
    total_loss = 0  # 新增：记录总损失
    for batch in loader:  # 批量处理
        optimizer.zero_grad()
        batch = batch.to(device)
        z = model.encode(batch.x, batch.train_pos_edge_index)  # 对 batch 进行编码
        # 计算重构损失和 KL 损失
        loss = model.recon_loss(z, batch.train_pos_edge_index)
        loss += (1 / batch.num_nodes) * model.kl_loss()  # 使用 batch.num_nodes 归一化 KL 损失
        loss.backward()  # 反向传播
        optimizer.step()  # 更新参数
        total_loss += loss.item()  # 累加每个批次的损失
    return total_loss / len(loader)  # 返回平均损失


def test():
    print('正在评估中...🀄️')
    model.eval()  # 设置模型为评估模式
    total_auc = 0  # 记录总 AUC
    total_ap = 0   # 记录总 AP
    valid_batches = 0  # 记录有效批次数量

    with torch.no_grad():  # 禁用梯度计算
        for batch in loader:  # 对每个 batch 进行测试
            batch = batch.to(device)

            # 检查 test_pos_edge_index 和 test_neg_edge_index 是否为空
            if batch.test_pos_edge_index.size(1) == 0 or batch.test_neg_edge_index.size(1) == 0:
                print("警告: test_pos_edge_index 或 test_neg_edge_index 为空，跳过该批次。")
                continue  # 跳过当前批次

            z = model.encode(batch.x, batch.train_pos_edge_index)  # 对 batch 进行编码
            
            # 使用测试集的正边和负边进行测试
            auc, ap = model.test(z, batch.test_pos_edge_index, batch.test_neg_edge_index)
            total_auc += auc
            total_ap += ap
            valid_batches += 1  # 增加有效批次计数

    # 返回平均 AUC 和 AP，如果没有有效批次，则返回 0
    return (total_auc / valid_batches) if valid_batches > 0 else 0.0, (total_ap / valid_batches) if valid_batches > 0 else 0.0



# 主训练循环
for epoch in range(1, epochs + 1):
    loss = train()  # 训练
    auc, ap = test()  # 测试
    print('Epoch: {:03d}, Loss: {:.4f}, AUC: {:.4f}, AP: {:.4f}'.format(epoch, loss, auc, ap))


正在评估中...🀄️
警告: test_pos_edge_index 或 test_neg_edge_index 为空，跳过该批次。
警告: test_pos_edge_index 或 test_neg_edge_index 为空，跳过该批次。
Epoch: 001, Loss: 1.5825, AUC: 0.3990, AP: 0.6279
正在评估中...🀄️
警告: test_pos_edge_index 或 test_neg_edge_index 为空，跳过该批次。
警告: test_pos_edge_index 或 test_neg_edge_index 为空，跳过该批次。
Epoch: 002, Loss: 1.5785, AUC: 0.3683, AP: 0.6096
正在评估中...🀄️
警告: test_pos_edge_index 或 test_neg_edge_index 为空，跳过该批次。
警告: test_pos_edge_index 或 test_neg_edge_index 为空，跳过该批次。
Epoch: 003, Loss: 1.5705, AUC: 0.3737, AP: 0.6132
正在评估中...🀄️
警告: test_pos_edge_index 或 test_neg_edge_index 为空，跳过该批次。
警告: test_pos_edge_index 或 test_neg_edge_index 为空，跳过该批次。
Epoch: 004, Loss: 1.5765, AUC: 0.3749, AP: 0.6138
正在评估中...🀄️
警告: test_pos_edge_index 或 test_neg_edge_index 为空，跳过该批次。
警告: test_pos_edge_index 或 test_neg_edge_index 为空，跳过该批次。
Epoch: 005, Loss: 1.5709, AUC: 0.3740, AP: 0.6128
正在评估中...🀄️
警告: test_pos_edge_index 或 test_neg_edge_index 为空，跳过该批次。
警告: test_pos_edge_index 或 test_neg_edge_index 为空，跳过该批次。
Epoch: 006, 

KeyboardInterrupt: 